In [1]:
from algothon2021long_short import LongShort
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
filenumbers = [11,12,13,14,15]
data = []
for i in filenumbers:
    # Load csv files
    df = pd.read_csv(f'long_short_training_features_targets/sector 3 - gold/Asset_{i}_training_data.csv')
    df['asset'] = i
    df = df.reset_index()
    data.append(df)

data = pd.concat(data, axis=0)

In [3]:
X = df.filter(like = 'log_returns')
X['intercept'] = 1.0
y = df['target']

<ipython-input-3-c1125f0bc344>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1.0


In [4]:
reg_model  = sm.OLS(y, X, hasconst = True)
fit = reg_model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.654e+31
Date:                Thu, 01 Apr 2021   Prob (F-statistic):               0.00
Time:                        20:26:31   Log-Likelihood:                 60460.
No. Observations:                1756   AIC:                        -1.209e+05
Df Residuals:                    1749   BIC:                        -1.209e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
log_returns_1    -1.54e-15   1.92e-16     -8.031      0.000   -1.92e-15   -1.16e-15
log_returns_5      -1.0000   1.23e-16  -8.15e+15      0.000      -1.000      -1.000
log_returns_10      1.0000   9.45e-17   1.06e+16      0.000       1.000       1.000
log_returns_25   1.659e-16    5.7e-17      2.909      0.004     5.4e-17    2.78e-16
log_returns_50   2.975e-16      4e-17      7.439      0.000    2.19e-16    3.76e-16
log_returns_100 -1.104e-16   2.16e-17     -5.104      0.000   -1.53e-16    -6.8e-17
intercept        2.201e-16   6.54e-18     33.679      0.000    2.07e-16    2.33e-16
==============================================================================
Omnibus:                        8.272   Durbin-Watson:                   0.382
Prob(Omnibus):                  0.016   Jarque-Bera (JB):                9.825
Skew:                          -0.078   Prob(JB):                      0.00735
Kurtosis:                       3.332   Cond. No.                         31.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
# filenumbers = [11,12,13,14,15]
# data = []
# for i in filenumbers:
#     # Load csv files
#     df = pd.read_csv(f'Asset_{i}_test_features.csv')
#     df['asset'] = i
#     df = df.reset_index()
#     data.append(df)

# data = pd.concat(data, axis = 0)

In [6]:
# Target is literally just log returns 10 subtracted by log returns 5
data['signal'] = data['log_returns_10'] - data['log_returns_5']
data[data['index'] == 0] # Show all data at day 0

,index,log_returns_1,log_returns_5,log_returns_10,log_returns_25,log_returns_50,log_returns_100,target,asset,signal
0,0,-0.012870,0.007569,0.072028,0.082662,0.014534,0.062498,0.064458,11,0.064458
0,0,-0.017628,0.004191,0.071789,0.044043,-0.052249,-0.005053,0.067598,12,0.067598
0,0,-0.022258,0.020234,0.078590,0.010168,-0.128155,-0.255704,0.058356,13,0.058356
0,0,-0.008653,0.009669,0.047961,0.008579,-0.088496,-0.183606,0.038292,14,0.038292
0,0,-0.029976,-0.009308,0.060386,-0.085486,-0.310129,-0.294849,0.069694,15,0.069694


In [7]:
data = data.reset_index(drop=True)

In [8]:
# Function to assign weights to signals
def make_long_short(signal):
    # Find mean signal of 5 assets and subtract all by this mean to give +ve and -ve values centred at 0
    signal -= np.mean(signal)
    # Take absolute maximum value of signal and divide by this value to place all values within the range of -1 to 1
    signal /= np.max(np.abs(signal))
    
    return signal

In [9]:
# Create 'weight' column, apply make_long_short function to signal column after grouping them by each day
data['weight'] = data.groupby('index')['signal'].apply(make_long_short)
data = data[['index', 'asset', 'weight']]

In [12]:
len(data)/5

1756.0

In [13]:
clean_data = []
# for i in number of days
for i in range(0, int(len(data)/5)):
    
    temp = data[data['index'] == i]
    temp = temp['weight'].values
    temp = pd.DataFrame(temp)
    df_transposed = temp.T
    clean_data.append(df_transposed)
    
clean_data = pd.concat(clean_data, axis=0)

In [17]:
final = pd.DataFrame(clean_data)
final = final.reset_index()
final = final.drop(['index'], axis=1)
# Export weights to csv
final.to_csv(f'LaPacks_test_long_short.csv', index=False)